In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
# from torch.distributed import

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


# DataLoader

ToDo: make dataloader

In [ ]:
class MyDataset(Dataset):

    def __init__(self, X, n=1000):
        self.X = torch.rand((X, n), dtype=torch.float64, device=device)
        self.y = torch.rand((X, 1), dtype=torch.float64, device=device)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        return (self.X[index], self.y[index])

In [ ]:
train_dataset = MyDataset(50_000)
test_dataset = MyDataset(10_000)
batch_size = 10_000
num_epochs = 10

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Model

In [ ]:
class PreliminaryModel(nn.Module):
    def __init__(self, n_feature):
        super().__init__()
        self.n_feature = n_feature
        self.seq_model = nn.Sequential(
            nn.Linear(self.n_feature, 2048, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(2048, 2048, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(2048, 2048, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(2048, 1024, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(1024, 1024, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(1024, 512, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(512, 512, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(512, 256, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(256, 128, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(128, 1, dtype=torch.float64),
            nn.Sigmoid()
        )

    def forward(self,x):
        y_pred = self.seq_model(x)
        return y_pred

In [ ]:
model = PreliminaryModel(1000).to(device)
model(torch.rand((5,1000), dtype=torch.double, device=device))

tensor([[0.4967],
        [0.4967],
        [0.4967],
        [0.4967],
        [0.4967]], device='cuda:0', dtype=torch.float64,
       grad_fn=<SigmoidBackward0>)

# Train

In [ ]:
def RMSELoss(y_pred, y):
    return torch.sqrt(torch.mean((y_pred-y)**2))

In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()

    running_loss = 0.0

    for inputs, labels in tqdm(train_loader):

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

In [ ]:
optimizer = opt.RMSprop(model.parameters(), lr=7e-4, alpha=0.99, eps=1e-5)
criterion = RMSELoss

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch) --

# Test

In [ ]:
def test(model, device, test_loader, criterion):

    model.eval()

    test_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
    print(f'Test Loss: {test_loss / len(test_loader):.4f}')

In [ ]:
for epoch in range(1, num_epochs + 1):
    test(model, device, test_loader, criterion)

100%|██████████| 5/5 [00:30<00:00,  6.01s/it]
